In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os

In [2]:
ledir = '/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/ice/proc/tseries/daily/aice_d/'

In [ ]:
%%time
listds = []
for e in range(1,36,1):
    ens = str(e)
    if e<10:
        ens = '0'+str(e)
    tmp = []
    print(ens)
    
    varlist = []
    for hemi in ['nh','sh']:
        if e==35: # this one seems to have a weird time coordinate
            ds = xr.open_mfdataset(ledir+'b.e11.B20TRC5CNBDRD.f09_g16.035.cice.h1.aice_d_'+hemi+'.19200101-20051231.nc').isel(time=slice(365*59-1,365*(59+22)-1))
        else:
            ds = xr.open_mfdataset(ledir+'b.e11.B20TRC5CNBDRD.f09_g16.0'+ens+'.cice.h1.aice_d_'+hemi+'.*-20051231.nc').sel(time=slice('1979-01','2000-12'))
        ints = ds.aice_d.copy()
        ints = ints.where(ints>15.)
        ints = ints.where(ints!=ints,1.)
        
        if hemi=='nh':
            mytarea = ds.tarea.where(ds.TLAT>0.)
        else:
            mytarea = ds.tarea.where(ds.TLAT<0.)
        
        sia = ((ds.aice_d/100.*mytarea).sum(dim=('ni','nj'))/1e12).to_dataset(name='sia_'+hemi).compute()
        sie = ((ints*mytarea).sum(dim=('ni','nj'))/1e12).to_dataset(name='sie_'+hemi).compute()
        varlist.append(sie)
        varlist.append(sia)
        

    ds = xr.merge(varlist)
    ds['names'] = 'LENS-'+ens
    ds = ds.set_coords('names')

    listds.append(ds)
    

01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [ ]:
lends = xr.concat(listds,dim='names')

In [ ]:
lends.sia_nh.attrs['units'] = '10^6 km^2'
lends.sie_nh.attrs['units'] = '10^6 km^2'
lends.sia_sh.attrs['units'] = '10^6 km^2'
lends.sie_sh.attrs['units'] = '10^6 km^2'
lends.attrs['processed'] = 'Lettie Roach, 2021'

In [ ]:
lends.to_netcdf('~/analysis/analysis_antarctic-asym/sea-ice-seasonality/netcdf/sie_sia_daily_LENS_1979-2000.nc')